In [1]:
import math


class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids


In [2]:
import cv2
from tracker import *

# Create tracker object
tracker = EuclideanDistTracker()

cap = cv2.VideoCapture(0)


object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape

    # Extract Region of interest
    roi = frame[340: 720,500: 800]

    # 1. Object Detection
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    for cnt in contours:

        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 100:
            #cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h = cv2.boundingRect(cnt)


            detections.append([x, y, w, h])

    # 2. Object Tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.putText(roi, str(id), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

    cv2.imshow("roi", roi)
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)

    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

{0: (25, 132)}
{7: (70, 21)}
{7: (70, 20)}
{7: (70, 20)}
{7: (70, 16)}
{7: (70, 14)}
{8: (90, 5), 9: (26, 10)}
{8: (90, 5), 9: (37, 6)}
{10: (98, 45), 11: (23, 39)}
{10: (122, 51), 11: (23, 39)}
{11: (25, 60), 10: (122, 51)}
{11: (28, 81), 12: (92, 67), 13: (25, 91)}
{15: (50, 131)}
{15: (47, 131), 16: (10, 96)}
{15: (47, 131), 16: (10, 101)}
{17: (25, 127)}
{17: (31, 130)}
{17: (38, 131), 18: (10, 83)}
{17: (38, 131), 18: (14, 93)}
{17: (42, 128), 18: (14, 93), 19: (10, 53)}
{17: (42, 128), 18: (20, 70), 19: (10, 53)}
{17: (44, 122), 18: (20, 70)}
{17: (44, 122), 18: (23, 70)}
{17: (46, 118), 18: (23, 70)}
{17: (46, 118), 18: (24, 71), 20: (54, 75)}
{17: (46, 118), 20: (54, 75), 18: (27, 95)}
{17: (48, 100), 20: (54, 75), 18: (27, 95)}
{18: (31, 94), 17: (48, 100), 21: (25, 59)}
{18: (48, 103), 17: (48, 100), 21: (25, 59)}
{18: (48, 103), 17: (48, 100), 21: (27, 56)}
{18: (31, 95), 21: (27, 56)}
{18: (48, 103), 21: (27, 56)}
{18: (48, 103), 21: (28, 56)}
{18: (30, 95), 21: (28, 56)}
{

{67: (75, 32), 68: (26, 14)}
{70: (78, 44)}
{75: (78, 9)}
{75: (85, 12), 76: (46, 48)}
{76: (48, 49), 75: (85, 12)}
{76: (48, 49), 75: (84, 13)}
{76: (52, 50), 75: (84, 13)}
{76: (52, 50), 75: (86, 14)}
{76: (49, 52), 75: (86, 14)}
{76: (49, 52), 75: (90, 15)}
{76: (51, 52), 75: (90, 15)}
{76: (51, 52), 75: (96, 16)}
{76: (51, 53), 75: (96, 16)}
{76: (51, 53), 75: (94, 15)}
{76: (56, 55), 75: (94, 15)}
{76: (56, 55), 75: (96, 17)}
{76: (56, 60), 75: (96, 17)}
{76: (56, 60), 75: (99, 19)}
{76: (51, 61), 75: (99, 19)}
{76: (51, 61), 75: (100, 19)}
{76: (51, 63), 75: (100, 19)}
{76: (51, 63), 75: (99, 21)}
{76: (51, 65), 75: (99, 21)}
{76: (51, 65), 75: (102, 23)}
{76: (51, 68), 75: (102, 23)}
{76: (51, 68), 75: (100, 25)}
{76: (53, 75), 75: (100, 25)}
{76: (53, 75), 75: (99, 29)}
{76: (53, 75), 75: (99, 34)}
{75: (100, 51)}
{75: (100, 57)}
{75: (100, 65), 77: (55, 128), 78: (38, 98)}
{77: (46, 134), 78: (38, 98), 75: (100, 65)}
{77: (46, 134), 78: (39, 101), 75: (100, 65)}
{77: (46, 134)